# Homework 1: Information Retrieval
## Instructions
1. Students will form teams of three people each and submit a single homework for each team in the format - ID1_ID2_ID3.ipynb
2. Groups of three!
2. **Do not write your names anywhere.**
3. For the code part: 
> **Write your code only in the mentioned sections. Do not change the code of other sections**. Do not use any imports unless we say so.
4. For theoretical questions, if any - write your answer in the markdown cell dedicated to this task, in **English**.


#### Deviation from the aforementioned  instructions will lead to reduced grade
---


## Clarifications
1. The same score for the homework will be given to each member of the team.  
2. The goal of this homework is to test your understanding of the concepts presented in the lectures. \
Anyhow, we provide here detailed explanations for the code part and if you have problems - ask.
3. Questions can be sent to the forum, you are encouraged to ask questions but do so after you have been thinking about your question. 
4. The length of the empty gaps (where you are supposed to write your code) is a recommendation (the amount of space took us to write the solution) and writing longer code will not harm your grade. We do not expect you to use the programming tricks and hacks we used to make the code shorter.   
Having said that, we do encourage you to write good code and keep that in mind - **extreme** cases may be downgraded.  
We also encourage to use informative variable names - it is easier for us to check and for you to understand. 

Since it is the first time we provide this homework please notify us if there is a bug/something is unclear, typo's exc..

5. We use Python 3.7 for programming.
6. Make sure you have all the packages and functions used in the import section. Most of it is native to Anaconda Python distribution.

### Have fun !

# Imports

In [1]:
import numpy as np
import pandas as pd
from collections import Counter
%matplotlib inline
from tqdm import tqdm
from typing import List,Dict
from IPython.display import Image
from IPython.core.display import HTML 
import time
import os
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer
import nltk
from string import punctuation, ascii_lowercase
from nltk.corpus import stopwords
from math import log, sqrt

nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tom\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tom\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
tic = time.perf_counter() #Start timer

# Config

In [3]:
cwd=os.getcwd()
INPUT_FILE_PATH = os.path.join(cwd, "lyrics.csv")
BOW_PATH = os.path.join(cwd, "bow.csv")
N_ROWS = 100000
CHUNCK_SIZE = 5000
tqdm_n_iterations = N_ROWS//CHUNCK_SIZE +1
COLS = [0]

## 1.1 Bag of words /TfIdf model
### Implement the following methods:

* `preprocess_sentence`: 
    * Lower case the word
    * Ignores it if it's in the stopwords list
    * Removes characters which are not in the allowed symbols
    * Stems it and appends it to the output sentence
    * Discards words with length <= 1
    
    
* `update_counts_and_probabilities`: 

    * Update self.unigram count (the amount of time each word is in the text)
    * Update self.bigram count (two consecutive word occurances)
    * Update inverted index: a dictionary with words as keys and the values is a dictionary - {'DocID' : word_count}   
    
* `compute_word_document_frequency`:

   * For each word count the number of docs it appears in. For example , for the word 'apple' -
$$\sum_{i \in docs} I(apple \in doc_i), I := Indicator function$$


* `update_inverted_index_with_tf_idf_and_compute_document_norm`:

    * Update the inverted index (which currently hold word counts) with tf idf weighing. We will compute tf by dividing with the number of words in each document. 
    * As we want to calculate the document norm, incrementally update the document norm. pay attention that later we apply sqrt to it to finish the process.

#### The result of this code is a bag of words model that already counts for TF-IDF weighing

In [4]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
allowed_symbols = set(l for l in ascii_lowercase)

In [5]:
def preprocess_sentence(sentence : str) -> List[str]:
    '''
    this function preprocesses a sentence by tokenizing it, removing stop words, stemming and removing punctuation and non-english words

    Parameters
    ----------
    sentence : str
        a string representing a sentence
    
    Returns
    -------
    List[str]
        a list of words
    '''
    output_sentence = []
    for word in word_tokenize(sentence):
        ### YOUR CODE HERE
     # Tokenize the sentence into words
        words = word_tokenize(sentence)

        # Remove stop words
        stop_words = set(stopwords.words('english'))
        words = [word for word in words if not word.lower() in stop_words]

        # Initialize the Porter stemmer
        porter = PorterStemmer()

        # Stem the words
        output_sentence = [porter.stem(word) for word in words]

        # Remove punctuation and non-English words
        output_sentence = [word.lower() for word in words if word.isalpha()]

        return output_sentence

            ### END YOUR CODE


def get_data_chuncks() -> List[str]:
    for i ,chunck in enumerate(pd.read_csv(INPUT_FILE_PATH, usecols = COLS, chunksize = CHUNCK_SIZE, nrows = N_ROWS)):
        chunck = chunck.values.tolist()
        yield [chunck[i][0] for i in range(len(chunck))] 

In [6]:
class TfIdf:
    def __init__(self):
        #unigram_count is a dictionary that maps a word to the number of times it appears in the corpus
        self.unigram_count =  Counter()
        #bigram_count is a dictionary that maps a bigram to the number of times it appears in the corpus
        self.bigram_count = Counter()
        #document_term_frequency is a dictionary that maps a document id to the number of words in the document
        self.document_term_frequency = Counter()
        #word_document_frequency is a dictionary that maps a word to the number of documents it appears in
        self.word_document_frequency = {}
        #inverted_index is a dictionary that maps a word to a dictionary that maps a document id to the tf-idf weight of the word in the document
        self.inverted_index = {}
        #doc_norms is a dictionary that maps a document id to the norm of the document
        self.doc_norms = {}
        #n_docs is the number of documents in the corpus
        self.n_docs = -1
        #sentence_preprocesser is a function that preprocesses a sentence
        self.sentence_preprocesser = preprocess_sentence
        #bow_path is the path to the bag of words
        self.bow_path = BOW_PATH
    
    def update_counts_and_probabilities(self, sentence:List[str], document_id:int) -> None:
        '''
        This function updates the unigram and bigram counts and the inverted index given a sentence and a document id

        Parameters
        ----------
        sentence : List[str]
            A list of words
        document_id : int
            The id of the document

        Returns
        -------
        None
        '''
        sentence_len = len(sentence)
        self.document_term_frequency[document_id] = sentence_len
        for i, word in enumerate(sentence):
            self.unigram_count[word] += 1
            if i > 0:
                bigram = sentence[i-1] + ' ' + word
                self.bigram_count[bigram] += 1
            if word not in self.word_document_frequency:
                self.word_document_frequency[word] = set()
            self.word_document_frequency[word].add(document_id)

        for word in set(sentence):
            if word not in self.inverted_index:
                self.inverted_index[word] = {}
            tf = 1 + log(sentence.count(word))
            df = len(self.word_document_frequency[word])
            if df < self.n_docs:
                idf = log(self.n_docs/df)
                tf_idf = tf * idf
                self.inverted_index[word][document_id] = tf_idf

        if document_id in self.inverted_index[word]:
            self.doc_norms[document_id] = sqrt(sum(pow(self.inverted_index[word][document_id], 2) for word in self.inverted_index if document_id in self.inverted_index[word]))

            ### END YOUR CODE

        
    def fit(self) -> None:
        for chunck in tqdm(get_data_chuncks(), total = tqdm_n_iterations):
            for sentence in chunck: #sentence is a song (string)
                self.n_docs += 1 
                if not isinstance(sentence, str):
                    continue
                sentence = self.sentence_preprocesser(sentence)
                if sentence:
                    self.update_counts_and_probabilities(sentence,self.n_docs)
        self.save_bow() # bow is 'bag of words'
        self.compute_word_document_frequency()
        self.update_inverted_index_with_tf_idf_and_compute_document_norm()

    def compute_word_document_frequency(self):
        '''
        Computes the word document frequency for each word in the inverted index. The word document frequency is the number of documents in the corpus that contain a particular word.

        Parameters
        ----------
        None

        Returns
        -------
        None
        '''
        for word in self.inverted_index.keys():
            self.word_document_frequency[word] = len(self.inverted_index[word].keys())


    def update_inverted_index_with_tf_idf_and_compute_document_norm(self):
        '''
        Updates the inverted index with tf-idf weighting and computes the document norm for each document.

        Parameters
        ----------
        None

        Returns
        -------
        None
        '''
        for word in self.inverted_index.keys():
            for doc_id in self.inverted_index[word].keys():
                tf = self.inverted_index[word][doc_id]
                idf = log(self.n_docs/self.word_document_frequency[word])
                tf_idf = tf * idf
                self.inverted_index[word][doc_id] = tf_idf
                if doc_id not in self.doc_norms:
                    self.doc_norms[doc_id] = 0.0
                self.doc_norms[doc_id] += pow(tf_idf, 2)

            for doc in self.doc_norms.keys():
                self.doc_norms[doc] = np.sqrt(self.doc_norms[doc]) 

    def save_bow(self):
        pd.DataFrame([self.inverted_index]).T.to_csv(self.bow_path)

In [7]:
tf_idf = TfIdf()
tf_idf.fit()

 95%|█████████▌| 20/21 [34:29<01:43, 103.49s/it]


## 1.11 Bag of words model:

1. What is the computational complexity of this model, as a function of the number of docs in the corpus?
2. How can we make this code better in terms running time (parallelization or other topics you find)? 

## 1.2 DocumentRetriever
Not this retriever &#8595;


![dsafdsafsdafdsf](https://cdn3-www.dogtime.com/assets/uploads/2019/10/golden-cocker-retriever-mixed-dog-breed-pictures-cover-1.jpg)

### Implement the following methods:

`reduce_query_to_counts`: given a list of words returns a counter object with words as keys and counts as values.

`rank`: given a query and relevant documents calculate the similarity (cosine or inner product simialrity) between each document and the query.   
Make sure to transform the query word counts to tf idf as well. 

`sort_and_retrieve_k_best`: returns the top k documents.

In [14]:
class DocumentRetriever:
    def __init__(self, tf_idf):
        #sentence_preprocesser is a function that preprocesses a sentence
        self.sentence_preprocesser = preprocess_sentence  
        #vocab is a set of all the words in the corpus
        self.vocab = set(tf_idf.unigram_count.keys())
        #n_docs is the number of documents in the corpus
        self.n_docs = tf_idf.n_docs
        #inverted_index is a dictionary that maps a word to a dictionary that maps a document id to the tf-idf weight of the word in the document
        self.inverted_index = tf_idf.inverted_index
        #word_document_frequency is a dictionary that maps a word to the number of documents it appears in
        self.word_document_frequency = tf_idf.word_document_frequency
        #doc_norms is a dictionary that maps a document id to the norm of the document
        self.doc_norms = tf_idf.doc_norms
        
    def rank(self, query:Dict[str,int], documents:Dict[str,Counter], metric:str) -> Dict[str, float]:
        '''
        this function ranks the documents according to the query and the chosen similarity metric

        Parameters
        ----------
        query : Dict[str,int]
            a dictionary of words and their counts in the query
        documents : Dict[str,Counter]
            a dictionary of documents that contain the query words and their tf-idf weights
        metric : str
            the similarity metric to use. can be 'cosine' or 'dot'
        
        Returns
        -------
        Dict[str, float]
            a dictionary of documents and their similarity to the query
        '''
        result = {} # key: DocID , value : float , simmilarity to query
        query_len = np.sum(np.array(list(query.values())))
        ### YOUR CODE HERE
        
        for doc_id, doc_counter in documents.items():
            # Compute the dot product or cosine similarity between the query and the document
        
            ### END YOUR CODE
            if metric == 'cosine':
                ### YOUR CODE HERE
                # Compute the cosine similarity between the query and the document
                dot_product = sum([query[word]*doc_counter[word] for word in query if word in doc_counter])
                query_norm = np.sqrt(sum([query[word]**2 for word in query]))
                doc_norm = np.sqrt(sum([doc_counter[word]**2 for word in doc_counter]))
                similarity = dot_product / (query_norm * doc_norm)
            elif metric == 'inner_product':
                similarity = sum([query[word]*doc_counter[word] for word in query if word in doc_counter])
                ### END YOUR CODE
            else:
                raise ValueError("Invalid metric. Must be 'cosine' or 'inner_product'.")
            result[doc_id] = similarity
        return result
        
    
    def sort_and_retrieve_k_best(self, scores:Dict[str, float], k:int)-> List[str]:
        '''
        this function sorts the documents according to their similarity to the query and returns the k best documents
        '''
        ### YOUR CODE HERE 
        return [x[0] for x in (sorted(scores.items(), key=lambda x: x[1], reverse=True))[:k]]
        ### END YOUR CODE

    
    def reduce_query_to_counts(self, query:List)->  Counter:
        '''
        this function reduces the query to a dictionary of words and their counts
        '''
        ### YOUR CODE HERE
        return Counter(query)
        ### END YOUR CODE
        
        
    def get_top_k_documents(self, query:str, metric:str , k=5) -> List[str]:
        query = self.sentence_preprocesser(query)
        query = [word for word in query if word in self.vocab] # filter nan 
        query_bow = self.reduce_query_to_counts(query)
        relavant_documents = {word : self.inverted_index.get(word) for word in query}
        ducuments_with_similarity = self.rank(query_bow,relavant_documents, metric)
        return self.sort_and_retrieve_k_best(ducuments_with_similarity,k)
        
dr = DocumentRetriever(tf_idf)

In [15]:
from IPython.display import HTML
query = "Better stop dreaming of the quiet life, 'cause it's the one we'll never know And quit running for that runaway bus 'cause those rosy days are few And stop apologizing for the things you've never done 'Cause time is short and life is cruel but it's up to us to change This town called malice"
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/KT6ZtUbVw1M?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

In [16]:
cosine_top_k = dr.get_top_k_documents(query, 'cosine')
print(cosine_top_k)
inner_product_top_k = dr.get_top_k_documents(query, 'inner_product')
print(inner_product_top_k)

['better', 'stop', 'dreaming', 'quiet', 'life']
['better', 'stop', 'dreaming', 'quiet', 'life']


for index, song in enumerate(pd.read_csv(INPUT_FILE_PATH,usecols = [0]).iloc[cosine_top_k]['Lyric']):
    sep = "#"*50
    print(F"{sep}\nsong #{index} \n{song} \n{sep}")

# 1.3 term statistics:
Use "tf_idf" object that we created earlier and answer the following questions:

1. How many unique words we have?
2. How many potential word bigrams we have? How many actual word bigrams we have? How do you explain this difference?
3. What is the storage size of the input file "lyrics.csv"? What is the output file (bow.csv) size? how do you explain this difference?  

In [33]:
# 1. 
### YOUR SOLUTION HERE
unique_words = len(tf_idf.word_document_frequency.keys())
print(unique_words)
### END YOUR SOLUTION

"""
### Your verbal solution here
The length of the frequency keys will determine the number of unique words
### End your verbal solution here
"""

# 2.
### YOUR SOLUTION HERE
potential_bigrams = unique_words ** 2
actual_bigrams = len(tf_idf.bigram_count.keys())
print(potential_bigrams)
print(actual_bigrams)
### END YOUR SOLUTION

"""
### Your verbal solution here
The potential number of bigrams is simply the square of the number of unique words.
The actual number of bigrams is the number of bigrams that appear in the corpus.

### End your verbal solution here
"""

# 3.
### YOUR SOLUTION HERE
input_file_size = os.path.getsize('lyrics.csv')
output_file_size = os.path.getsize('bow.csv')
print(input_file_size)
print(output_file_size)
### END YOUR SOLUTION

"""
### Your verbal solution here
The difference in file sizes between the input and output files can be explained by the fact that the 
input file contains raw text data, while the output file contains preprocessed and condensed information 
in the form of the bag-of-words representation
### End your verbal solution here
"""

110681
12250283761
2905054
168274338
194019616


'\n### Your verbal solution here\nThe difference in file sizes between the input and output files can be explained by the fact that the \ninput file contains raw text data, while the output file contains preprocessed and condensed information \nin the form of the bag-of-words representation\n### End your verbal solution here\n'

## 1.4 NgramSpellingCorrector
Now we will implement a Ngarm (character Ngrams) spelling corrector. That is, we have an out of vocabulary word (v) and we want to retrieve the most similar words (in our vocabulary) to this word.
we will model the similarity of two words by-

$$JaccardIndex =  \frac{|X \cap Y|}{|X \cup Y|}$$

Where v is an out of vocabulary word (typo or spelling mistake), w is in a vocabulary word, X is the ngram set of v and Y is the ngram set of w.
For example, if n == 3, the set of ngrams for word "banana" is set("ban","ana","nan","ana") = {"ban","ana","nan"}

In order to do it efficently, we will first construct an index from the possible Ngrams we have seen in our corpus to the words that those Ngrams appear in, in order prevent comparing v to all of the words in our corpus.
Then, we will implement a function that computes this similarity.

* Make sure you compute the JaccardIndex efficently!

In [16]:
def get_bigrams(word):
    for ngram in nltk.ngrams(word, 2):
        yield "".join(list(ngram))
    
"""
for example - get_bigrams is a generator, which is an object we can loop on:
for ngram in get_bigrams(word):
    DO SOMETHING
"""

'\nfor example - get_bigrams is a generator, which is an object we can loop on:\nfor ngram in get_bigrams(word):\n    DO SOMETHING\n'

In [17]:
class NgramSpellingCorrector:
    def __init__(self, unigram_counts: Counter, get_n_gram: callable):
        #unigram_counts is a dictionary that maps a word to its count in the corpus
        self.unigram_counts = unigram_counts
        #ngram_index is a dictionary that maps an ngram to a set of words that contain it
        self.ngram_index = {}
        #get_n_grams is a function that returns the ngrams of a word
        self.get_n_grams = get_n_gram
    
    def build_index(self) -> None:
        '''
        this function builds an index of ngrams to words that contain them
        the structure of the index is a dictionary of ngrams to sets of words
        '''
        ### YOUR CODE HERE
        
        for word in self.unigram_counts.keys():  # For all words in the unigram dictionary
            word_ngrams_set=set(self.get_n_grams(word))  # Split the word into letter pairs by the get_n_gram function (remove duplicate)
            for ngram in word_ngrams_set:   # Go over all the ngrams
                if ngram not in self.ngram_index:  # If ngram is not appended to the ngram_index dictionary 
                    self.ngram_index[ngram]=[]  # Initilize the value of this ngram as empty list
                self.ngram_index[ngram].append(word)  # For each ngram (key) append to it's value the word which certainly contains this ngram
       
        ### END YOUR CODE
        
    def get_top_k_words(self,word:str,k=5) -> List[str]:
        '''
        this function returns the k most probable candidate words for a given out of vocabulary word

        Parameters
        ----------
        word : str
            the out of vocabulary word
        k : int, optional
            the number of candidates to return, by default 5
        
        Returns
        -------
        List[str]
            a list of the k most probable candidate words
        '''
        ### YOUR CODE HERE

        probabilties={}  # Initilize dictionary which contains the probabilties of similarity for each word
        word_ngrams_set=set(self.get_n_grams(word)) # Split the word into letter pairs by the get_n_gram function (remove duplicate)
        for ngram in word_ngrams_set:  # Go over all the word's ngrams
            if ngram in self.ngram_index:  # If ngram in dictionary
                for word in self.ngram_index[ngram]:  # Go over each word that contain this ngram
                    if word not in probabilties: # If the word is not appears
                        split_word=set(self.get_n_grams(word))  # Split the word into letter pairs by the get_n_gram function (remove duplicate)
                        intersection = len(list(set(word_ngrams_set).intersection(split_word)))  # Calculated the intersection of ngrams between the two words 
                        union = (len(word_ngrams_set) + len(split_word)) - intersection  # Calculated the union of ngrams between the two words 
                        similarity=(float(intersection)/union)  # Calculated the similarity between the two words
                        probabilties[word]=similarity  # Append the similarity to the dictionary under the word
        return (list(dict(Counter(probabilties).most_common(k)).keys()) , probabilties) # Return the top K words with the highest similarity and the probabilities of all words
        
        ### END YOUR CODE


class BigramSpellingCorrector(NgramSpellingCorrector):
    def __init__(self, unigram_counts: Counter):
        super().__init__(unigram_counts, get_bigrams)

Code Understanding Question:

In the "get top k words" function, does each candidate word have repeat ones or there is a channce that the same word will be iterated more than once? Explain your answer.

Answer here:

In [18]:
out_of_vocab_word = 'acress'
bigram_spelling_corrector = BigramSpellingCorrector(tf_idf.unigram_count)
bigram_spelling_corrector.build_index()
candidate_words, scores = bigram_spelling_corrector.get_top_k_words(out_of_vocab_word)

In [19]:
candidate_words

['acres', 'cress', 'cresses', 'recress', 'massacres']

In [20]:
for word in candidate_words:
    print(f'The jaccared score for the word {word} is {round(scores[word],2)}')

The jaccared score for the word acres is 0.8
The jaccared score for the word cress is 0.8
The jaccared score for the word cresses is 0.67
The jaccared score for the word recress is 0.67
The jaccared score for the word massacres is 0.62


# The End - You did it!

In [2]:
#The time it took to run the entire code
toc = time.perf_counter()

print(f"The time it took to run the entire code is: {(toc - tic)/60} minuts")

The time it took to run the entire code is: 1907 minuts